In [2]:
import mesa_web as mw
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os 
import re

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, models, Sequential, Input
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from scipy.interpolate import UnivariateSpline

cwd=os.getcwd()
dir_names=['MESA-Web_M07_Z00001','MESA-Web_M1_Z0001']
column_filter = ['mass','radius', 'initial_mass', 'initial_z', 'star_age', 'logRho','logT','Teff','energy','photosphere_L', 'photosphere_r', 'star_mass','h1','he3','he4']
column_filter_train = ['mass','radius', 'logRho','logT','energy','h1','he3','he4']
n_points=50
r=np.linspace(0,1,n_points)


for i,dir_name in enumerate(dir_names):

  print(f"####\t\tIMPORTING DATA FROM FOLDER {dir_name}\t\t####")
  dir_name=os.path.join(cwd,'StellarTracks',dir_name)

  def extract_number(filename):
    match = re.search(r'\d+', filename)  #find the sequence of digits
    return int(match.group()) if match else float('inf')

  filenames=[filename for filename in os.listdir(dir_name) if re.fullmatch('profile[0-9]+\.data',filename)]
  filenames=sorted(filenames, key=extract_number) #sort the elements according to the number in the name

  for j,filename in enumerate(filenames):

    print(f"####\t\t\tIMPORTING FILE {filename}\t\t\t####")  
    filename=os.path.join(dir_name,filename)

    data=mw.read_profile(filename)

    profile_df=pd.DataFrame(data)

    # Create a new DataFrame with only the selected columns
    filtered_profile_df = profile_df[column_filter].copy()
    # Create a new DataFrame with only the selected columns for autoencoder training
    train_filtered_profile_df = profile_df[column_filter_train].copy()

    #Normalization process
    tot_radius=filtered_profile_df['photosphere_r']

    norm_radius=(filtered_profile_df['radius']-filtered_profile_df['radius'].min())/(tot_radius-filtered_profile_df['radius'].min())

    norm_radius=np.asarray(norm_radius.T)
    log_rho=np.asarray(train_filtered_profile_df['logRho'].T)
    int_log_rho=UnivariateSpline(norm_radius,log_rho,k=2,s=0)(r)

    train_df=pd.DataFrame(data=int_log_rho.T,columns=[f"log_rho_{i}_{j}"])  #in the format _indexOfFolder_IndexOfProfile
  
    if (i==0 and j==0):
      linear_train_df=train_df
    else:
      linear_train_df=pd.concat([linear_train_df,train_df],axis=1)

#print(linear_train_df.shape)
#print(linear_train_df)

x_train, x_test = train_test_split(linear_train_df.T, test_size=0.2)
print("Train:", x_train.shape)
print("Test:", x_test.shape)
#print(x_train)
#print(x_test)

x_train = tf.reshape(x_train, (x_train.shape[0], 50, 1))
x_test = tf.reshape(x_test, (x_test.shape[0], 50, 1))

print("Sos:", x_train.shape[1:])

<>:34: SyntaxWarning: invalid escape sequence '\.'
<>:34: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_52985/396764895.py:34: SyntaxWarning: invalid escape sequence '\.'
  filenames=[filename for filename in os.listdir(dir_name) if re.fullmatch('profile[0-9]+\.data',filename)]
2024-05-23 12:58:14.567514: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 12:58:14.619323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 12:58:15.664323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Wa

####		IMPORTING DATA FROM FOLDER MESA-Web_M07_Z00001		####
####			IMPORTING FILE profile1.data			####
####			IMPORTING FILE profile2.data			####
####			IMPORTING FILE profile3.data			####
####			IMPORTING FILE profile4.data			####
####			IMPORTING FILE profile5.data			####
####			IMPORTING FILE profile6.data			####
####			IMPORTING FILE profile7.data			####
####			IMPORTING FILE profile8.data			####
####			IMPORTING FILE profile9.data			####
####			IMPORTING FILE profile10.data			####
####			IMPORTING FILE profile11.data			####
####			IMPORTING FILE profile12.data			####
####			IMPORTING FILE profile13.data			####
####			IMPORTING FILE profile14.data			####
####			IMPORTING FILE profile15.data			####
####			IMPORTING FILE profile16.data			####
####			IMPORTING FILE profile17.data			####
####			IMPORTING FILE profile18.data			####
####			IMPORTING FILE profile19.data			####
####			IMPORTING FILE profile20.data			####
####			IMPORTING FILE profile21.data			####
####			IMPORTING FILE prof

2024-05-23 12:58:30.646378: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-23 12:58:30.647043: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
